So these are methods I tried to get working, but for some reasons or other they just would not. Maybe I'll still try to fine tune them to get them working in the future. But for now I decided to essentially 'downscale' what I was intending.  
I feel like I just tried biting off more than I could chew, especially with trying to have the multiRNN methods calling the singleRNN methods (almost stubbornly), the plotting just got complicated.

# Removed methods for singleRNN

In [ ]:

    def plot_metric(self, metric: str = 'loss', figsize: tuple = (5, 5), title: str = "auto", ax=None):
        if title == 'auto':  # Just to make a title with metric contained in it.
            title = f'Plot of {metric}'

        if ax is None:
            fig, ax = plt.subplots(figsize=figsize)
            own_fig = True
        else:
            own_fig = False

        try:
            if f'val_{metric}' in self.model_loss.history:
                ax.plot(self.model_loss.history[metric], label=metric)
                ax.plot(self.model_loss.history[f'val_{metric}'], label=f'val_{metric}')
            else:
                ax.plot(self.model_loss.history[metric], label=metric)
            ax.legend()
        except Exception as e:
            raise ValueError(f"An error occurred while plotting the metric, {metric}: {e}")

        ax.set_xlabel('epochs')
        ax.set_title(title)

        if own_fig:
            plt.show()

        return ax

    def plot_prediction_w_test(self, predictions:pd.DataFrame, x_range:tuple, columns:list = 'auto',
                               figsize:tuple = (5, 5), title:str = "Plot predicted vs test"):
        if columns == 'auto':
            columns = self.column_names
        SingleRNN._validate_columns(columns, self.column_names)
        
        fig, ax = plt.subplots(figsize=figsize)
        
        for column in columns:
            self.train[column].plot(ax=ax, label=f'Train {column}')
            self.test[column].plot(ax=ax, label=f'Test {column}')
            predictions[column].plot(ax=ax, label=f'Predictions {column}')

        ax.set_xlim(x_range)
        ax.set_title(title)
        ax.legend()
        plt.show()

    # I needed these last two methods to viably even test anything after training, minor changes requiring a full model re-training was 
    # just a waste of time.
    def save_singlernn(self, filename: str):
        # Check if the filename contains any file extension
        if re.search(r'\.[^.]+$', filename):
            raise ValueError("Error: Filename should not contain any file extension.")
        try:
            # Save the model to an .h5 file
            model_filename = filename + '_model.h5'
            self.model.save(model_filename)

            # Temporarily remove the model attribute for pickling
            model_backup = self.model
            self.model = None

            # Save the rest of the object using pickle
            with open(filename + '.pkl', 'wb') as f:
                pickle.dump(self, f)

            # Restore the model attribute
            self.model = model_backup

            print("Saved successfully.")
        except Exception as e:
            raise ValueError(f"An error occurred (save model): {e}")

    @staticmethod
    def load_singlernn(filename: str):
        if re.search(r'\.[^.]+$', filename):
            raise ValueError("Error: Filename should not contain any file extension.")
        try:
            # Load the main object using pickle
            with open(filename + '.pkl', 'rb') as f:
                obj = pickle.load(f)
            
            # Load the model from its .h5 file
            model_filename = filename + '_model.h5'
            obj.model = tf.keras.models.load_model(model_filename)

            print("Loaded successfully.")
            return obj
        except Exception as e:
            raise ValueError(f"An error occurred (load model): {e}")


# Removed methods for multiRNN

In [ ]:

    def plot_metric(self, columns, metric: str = 'loss', figsize: tuple = (5, 5), title: str = 'auto'):
        SingleRNN._validate_columns(columns, self.column_names)

        num_columns = len(columns)
        fig, axes = plt.subplots(num_columns, 1, figsize=(figsize[0], figsize[1] * num_columns), sharex=True)

        if num_columns == 1:
            axes = [axes]  # Ensure axes is iterable if there's only one subplot

        if title == 'auto':  # Just to make a title with metric contained in it.
            title = f'Plot of {metric} for {", ".join(columns)}'

        try:
            for i, column in enumerate(columns):
                index = self._find_column(column)
                self.multi_models[index].plot_metric(metric, figsize, ax=axes[i], title = None)
        except Exception as e:
            raise ValueError(f"An error occurred while plotting the metric, {metric}: {e}")

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to make space for super title
        fig.suptitle(title, fontsize=16)  # Add the super title here
        plt.show()

    def plot_prediction_w_test(self, predictions:pd.DataFrame, x_range:tuple, columns:list = 'auto',
                               figsize:tuple = (10, 6), title:str="Plot predicted vs test"):
        if columns == 'auto':
            columns = self.column_names
        SingleRNN._validate_columns(columns, self.column_names)

        num_columns = len(columns)
        fig, axes = plt.subplots(num_columns, 1, figsize=(figsize[0], figsize[1] * num_columns), sharex=True)

        if num_columns == 1:
            axes = [axes]  # Ensure axes is iterable if there's only one subplot

        try:
            for i, column in enumerate(columns):
                idx = self._find_column(column)
                axes[i].plot(self.train[:, idx], label=f'Train {column}')
                len_train = len(self.train_dfs[0])
                len_test = len(self.test_dfs[0])
                axes[i].plot(range(len_train, len_train + len_test), self.test[:, idx], label=f'Test {column}')
                axes[i].plot(range(len_train + len_test, len_train + len_test + len(predictions[0])), predictions[:, idx], label=f'Predictions {column}')
                axes[i].set_xlim(x_range)
                axes[i].legend()
        except Exception as e:
            raise ValueError(f"An error occurred while plotting predictions vs test for column '{column}': {e}")
        
        fig.suptitle(title, fontsize=16)
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()

    # I needed these last two methods to viably even test anything after training, minor changes requiring a full model re-training 
    # was just a waste of time.
    def save_multirnn(self, filename: str):
        # Check if the filename contains any file extension
        if re.search(r'\.[^.]+$', filename):
            raise ValueError("Error: Filename should not contain any file extension.")
        
        model_filenames = []
        try:
            # Save each model to an .h5 file
            for i, singlernn in enumerate(self.multi_models):
                model_filename = f"{filename}_model_{i}"
                singlernn.save_singlernn(model_filename)
                model_filenames.append(model_filename)

            # Remove the models temporarily
            models_backup = self.multi_models
            self.multi_models = model_filenames

            # Save the rest of the object using pickle
            with open(filename + '.pkl', 'wb') as f:
                pickle.dump(self, f)

            # Restore the models back to the object
            self.models = models_backup

            print("Saved successfully.")
        except Exception as e:
            raise ValueError(f"An error occurred (save model): {e}")

    @staticmethod
    def load_multirnn(filename: str):
        if re.search(r'\.[^.]+$', filename):
            raise ValueError("Error: Filename should not contain any file extension.")

        try:
            # Load the main object using pickle
            with open(filename + '.pkl', 'rb') as f:
                obj = pickle.load(f)
            
            # Load each model from its .h5 file
            model_filenames = obj.multi_models
            obj.multi_models = [SingleRNN.load_singlernn(model_filename) for model_filename in model_filenames]

            print("Loaded successfully.")
            return obj
        except Exception as e:
            raise ValueError(f"An error occurred (load model): {e}")
